# Спайковые нейронные сети

### Строение нейрона

<img src="img/natural_neuron.jpg" width="600px">

### Искусственные нейронные сети

1943 — У. Маккалок и У. Питтс формализуют понятие нейронной сети в фундаментальной статье о логическом исчислении идей и нервной активности.
Минусы идеи:
- исследования проводились на нейронах спинного мозга, тогда как нейроны головного мозга (пирамидельные нейроны) устроены несколько иначе;
- предлагалось проектировать нейронные сети поэлементно;
- построенная модель является значительным упрощением модели, которую мы с вами сейчас рассмотрим.

### Модели нейронов головного мозга

Альтернативные модели, которые мы не будем рассматривать:
- модель Ходжкина — Хаксли (модель электрического прохождения сигналов);
- модель ФитцХью — Нагумо (моделирование прохождения сигналов при помощи дифференциальных уравнений);
- кабельная теория дендритов (моделирование прохождение сигнала через разветвляющиеся дендриты);
- модель нейрона Ижикевича (модель разницы потенциалов на внутренней и внешней сторонах мембраны; наиболее близка к тому, что будет излагаться ниже; компромисс между биологической и математической точностью).

<img src="img/neuron_ions.png" width="400px">


### Краткое описание модели

Будем считать, что состояние нейрона определяется двумя переменными: значением активации и значением торможения. Если значение активации меньше торможения, нейрон находится в спокойном состоянии и не генерирует спайков на выходе. Если значение активации больше торможения, нейрон активен и генерирует спайки. Интервал между соседними спайками тем меньше, чем больше разница между уровнями активации и торможения.

Входы нейрона можно разделить на те, что повышают значения, и те, что понижают. Также входы делятся на активирующие (изменяют значение активации) и тормозящие (меняют значения торможения). Таким образом, входны нейрона можно разделить на четыре группы.

У уровней активации и торможения имеются пороговые значения, выше которых они не поднимаются. Увеличение и уменьшение значений зависит от разницы между текущим значением и максимальным. То есть чем ближе значение находится к своему максимуму, тем меньше будет подниматься значение активации или торможения. Таким образом, при большой частоте поступающих спайков на данный вход, значение всё равно не поднимется выше максимального. 

Если спайки не поступают на вход, значение по экспоненте стремится вернуться к значению покоя. В зависимости от частоты поступающих сигналов может наступать насыщение, когда изменение значения за счет поступающих спайков в промежутках компенсируется затуханием (возвращением к значению покоя).

В общем случае, у нейрона может быть много входов каждого типа. Каждому из таких входов присваивается собственное значение, на которое меняется активация или торможение нейрона. При поступлении нескольких спайков на разные входы, эффект будет складываться или вычитаться, в зависимости от типов входов.

In [1]:
import numpy as np
import matplotlib.pyplot as plt

plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (12,8)
from ipywidgets import Button, interact, interactive_output, HBox, VBox, Layout, Output
from IPython.display import display
import seaborn as sns

In [14]:
class Neuron:
    
    def __init__(self):
        self.I0 = 3.
        self.R0 = 10.
        self.I = self.I0
        self.R = self.R0
        self.I_max = 20.
        self.R_max = 16.
        self.interval = -1.
        self.time_to_spike = -1.
        self.spike = 0
        
    def __repr__(self):
        return f"Neuron: I - {self.I:5.4}, R - {self.R:5.4}, Interval - {self.interval:5.4},"\
               f" Interval - {self.time_to_spike:5.4}, Spike - {self.spike}"
        
    def spikeIPlus(self):
        self.I += 0.4 * (self.I_max - self.I)
    
    def spikeIMinus(self):
        self.I -= 0.2 * self.I

    def spikeRPlus(self):
        self.R += 0.5 * (self.R_max - self.R)
    
    def spikeRMinus(self):
        self.R -= 0.3 * self.R
        
    def calcInterval(self):
        if self.I <= self.R:
            self.interval = -1
            self.time_to_spike = -1
        else:
            if self.interval == -1:
                self.interval = 10 / (self.I - self.R)
                self.time_to_spike = self.interval
            else:
                self.interval = 10 / (self.I - self.R)
                
    def updateSpike(self):
        if self.interval != -1:
            self.time_to_spike -= 1
            if self.time_to_spike < 0:
                self.spike = 1
                self.time_to_spike += self.interval
            else:
                self.spike = 0
                
    def step(self):
        self.I -= (self.I - self.I0) * 0.1
        self.R -= (self.R - self.R0) * 0.1
        self.calcInterval()
        self.updateSpike()
        
    

In [15]:
def drawAN():
    ax.clear()
#    ax.plot(np.random.randn(100),np.random.randn(100),'+')
    hundred = list(range(100))
#     fig, axes = plt.subplots(1, 1, figsize=(14,7))
    #axes.clear()
    ax.plot(hundred, Is)
    ax.plot(hundred, Rs)
    ax.plot(hundred, sp)
    with out:
        out.clear_output(wait=True)
        display(ax.figure)
    
def click(b):
    global Is, Rs, sp, n
    
    n.step()
    Is = Is[1:]
    Is = np.append(Is, n.I)
    Rs = Rs[1:]
    Rs = np.append(Rs, n.R)
    sp = sp[1:]
    sp = np.append(sp, n.spike*10)
    drawAN()
    
def IPlusClicked(b):
    global Is, Rs, sp, n
    n.spikeIPlus()
    n.step()
    Is = Is[1:]
    Is = np.append(Is, n.I)
    Rs = Rs[1:]
    Rs = np.append(Rs, n.R)
    sp = sp[1:]
    sp = np.append(sp, n.spike*10)
    drawAN()

def IMinusClicked(b):
    global Is, Rs, sp, n
    n.spikeIMinus()
    n.step()
    Is = Is[1:]
    Is = np.append(Is, n.I)
    Rs = Rs[1:]
    Rs = np.append(Rs, n.R)
    sp = sp[1:]
    sp = np.append(sp, n.spike*10)
    drawAN()

def RPlusClicked(b):
    global Is, Rs, sp, n
    n.spikeRPlus()
    n.step()
    Is = Is[1:]
    Is = np.append(Is, n.I)
    Rs = Rs[1:]
    Rs = np.append(Rs, n.R)
    sp = sp[1:]
    sp = np.append(sp, n.spike*10)
    drawAN()

def RMinusClicked(b):
    global Is, Rs, sp, n
    n.spikeRMinus()
    n.step()
    Is = Is[1:]
    Is = np.append(Is, n.I)
    Rs = Rs[1:]
    Rs = np.append(Rs, n.R)
    sp = sp[1:]
    sp = np.append(sp, n.spike*10)
    drawAN()

def resetClicked(b):
    global Is, Rs, sp, n
    
    n = Neuron()
    Is = np.ones(100)
    Is *= n.I0
    Rs = np.ones(100)
    Rs *= n.R0
    sp = np.zeros(100)
    drawAN()
    
    
plt.ioff()

out=Output(layout=Layout(height='400px', width = '600px', border='solid'))
ax=plt.gca()

button1=Button(description='Step')
button2=Button(description='I+')
button3=Button(description='I-')
button4=Button(description='R+')
button5=Button(description='R-')
buttonReset=Button(description='Reset')
vbox=VBox(children=(button1, button2, button3, button4, button5, buttonReset))
hbox=HBox(children=(out,vbox))
display(hbox)
with out:
    out.clear_output(wait=True)

button1.on_click(click)
button2.on_click(IPlusClicked)
button3.on_click(IMinusClicked)
button4.on_click(RPlusClicked)
button5.on_click(RMinusClicked)
buttonReset.on_click(resetClicked)
resetClicked(None)

In [16]:
n

Neuron: I -   3.0, R -  10.0, Interval -  -1.0, Interval -  -1.0, Spike - 0